In [16]:
import sys
sys.path.insert(0,'/home/leo/Documents/cosmic_ray_generator/')
import functions
from api.rejection_sampler import generator
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize


Code to make the generator generate muons. This is the right code to be used if the generator will be incorporated into a python program

In [8]:
gen = generator(times=True,with_angles=True,detector_area=900)

In [17]:
a=gen.create(20000)

Bins the generated data

In [12]:
def binn(data,bins):
    b=[]
    for i in range(len(bins)-1):
        a=[]
        for j in data:
            if j<=bins[i+1] and j>=bins[i]:
                a.append(j)
        b.append(a)
    return b

Bins the data without any filtering. Uses the log of energy for the bins

In [87]:
bins=[-2+0.1*i for i in range(60)]
binned=binn([numpy.log10(a[i][0]) for i in range(len(a))],bins)
d=binned
c=pd.DataFrame({'freq':[len(d[i]) for i in range(len(d))],'p_average':[numpy.log10(numpy.mean([10**d[i][j] for j in range(len(d[i]))])) for i in range(len(d))],\
    'p_range':[10**bins[i+1]-10**bins[i] for i in range(len(bins)-1)]})
c=c.drop([i for i in range(len(c['freq'])) if c['freq'][i]<5])

/home/leo/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/leo/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Filters muons between certain angles and bins their angular distribution

In [55]:
vertical_c=[a[i][0] for i in range(len(a)) if a[i][1]<numpy.radians(34) and a[i][1]>numpy.radians(24)]
bins=[-3+0.1*i for i in range(60)]
d=binn([numpy.log10(vertical_c[i]) for i in range(len(vertical_c))],bins)
c=pd.DataFrame({'freq':[len(d[i]) for i in range(len(d))],'p_average':[numpy.log10(numpy.mean([10**d[i][j] for j in range(len(d[i]))])) for i in range(len(d))],\
    'p_range':[10**bins[i+1]-10**bins[i] for i in range(len(bins)-1)]})
c=c.drop([i for i in range(len(c['freq'])) if c['freq'][i]<1])

/home/leo/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/leo/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Finds the total number of muons remaining after filtering

In [45]:
numpy.sum([len(d[i]) for i in range(len(d))])

2635

Filters muons by energy and bins the remaining muons by angle

In [13]:
angle_c=[a[i][1] for i in range(len(a)) if a[i][0]>0.6]
bins=[numpy.deg2rad(5*i) for i in range(0,18)]
d=binn(angle_c, bins)
c=pd.DataFrame({'freq':[len(d[i]) for i in range(len(d))],'zenith angle':[numpy.rad2deg((bins[i]+bins[i+1])/2) for i in range(17)]})

In [14]:
c

,freq,zenith angle
0,17966,2.5
1,17766,7.5
2,17400,12.5
3,16892,17.5
4,16117,22.5
5,15352,27.5
6,14138,32.5
7,12869,37.5
8,11663,42.5
9,10167,47.5


Estimates a differential intensity for a specific experiment

In [89]:
c['diff_int']=c['freq']/(c['p_range']*(2.56*10**4*4*numpy.pi*(1-numpy.cos(numpy.deg2rad(1)))))
c['p_average']=10**c['p_average']
c

,freq,p_average,p_range,diff_int
43,77167,222.585221,51.662412,30.485505
44,48398,279.950339,65.039123,15.187602
45,29849,352.478114,81.879405,7.440319
46,18075,443.637837,103.080063,3.578822
47,10768,558.338033,129.770111,1.693545
48,6402,702.387415,163.370890,0.799793
49,3939,884.589496,205.671765,0.390884
50,2267,1110.268403,258.925412,0.178695
51,1366,1398.370317,325.967781,0.085529
52,756,1761.681074,410.369123,0.037600


Generates a plot of the data generated

In [ ]:
plt.scatter(c['p_average'],c['diff_int']*(c['p_average'])**3, marker='x')
plt.yscale('log')
plt.xscale('log')
plt.show()

Saves the generator output so it can be used by other programs

In [15]:
with open('gen_data_angle.npy','wb') as f:
    numpy.save(f,c)
c

,freq,zenith angle
0,17966,2.5
1,17766,7.5
2,17400,12.5
3,16892,17.5
4,16117,22.5
5,15352,27.5
6,14138,32.5
7,12869,37.5
8,11663,42.5
9,10167,47.5


Code for a least squares fit of a sec curve

In [48]:
def exp(x, params):
    a=params
    return a/(numpy.cos(numpy.deg2rad(x)))
def res(params,x,y):
    return y-exp(x,params)
def fit(data):
    popt=optimize.leastsq(res,x0=numpy.array([100]),args=(data[0],data[1]))
    print(popt)
    return(popt[0])

In [ ]:
x=numpy.linspace(0,80,10000)
plt.scatter(c['zenith angle'],c['freq'], marker='x', label='generator data', color='black')
# p=fit([c['zenith angle'],c['freq']])
# p1=p[0]
# p2=p[1]
# y=p1*numpy.cos(numpy.deg2rad(x))**p2
# plt.plot(x, y, color='m', label='cos^2.19')
# plt.plot(x,p1*numpy.cos(numpy.deg2rad(x))**2, label='cos^2', color='r')
p=fit([c['zenith angle'],c['freq']])
plt.plot(x,500/(numpy.cos(numpy.deg2rad(x))), label='sec')
#plt.plot(x, 1/numpy.cos(numpy.deg2rad(x)))
plt.legend()